In [79]:
import wandb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [80]:
# Initialize the API
api = wandb.Api()

# Specify your project details
entity = "haeri-hsn"  # Replace with your wandb entity
project = "stream_learning"  # Replace with your wandb project name

# Define filters
filters = {
    'state': 'finished',  # Only fetch finished runs
    'tags': {'$in': ['teconer_final']}  # Runs containing a specific tag msmsa_horizon_analysis_melbourne_housing \ msmsa_anchor_analysis_melbourne_housing
}

# Query runs with filters
runs = api.runs(f"{entity}/{project}", filters=filters)

# print number of runs
print(f"Number of runs: {len(runs)}")

Number of runs: 343


In [86]:
# Extract data from each run
summary_list = []
config_list = []
name_list = []
for run in runs:
    # run.summary contains the output of the training
    summary_list.append(run.summary._json_dict)

    # run.config contains the hyperparameters
    config_list.append({k: v for k, v in run.config.items() if not k.startswith('_')})

    # run.name is the name of the run.
    name_list.append(run.name)

# Convert to DataFrame
summary_df = pd.DataFrame.from_records(summary_list)
config_df = pd.DataFrame.from_records(config_list)
name_df = pd.DataFrame({'name': name_list})

# Combine all parts into one DataFrame
df = pd.concat([name_df, config_df, summary_df], axis=1)
# remove the columns that are repeated
df['preview_duration'] = df['preview_duration'].astype('float')/60
df = df.loc[:, ~df.columns.duplicated()]
# change the name of the metrics and remove metric_ from the name
df = df.rename(columns=lambda x: x.replace("metric_", ""))
# Display the DataFrame
print(df.columns)

# print the values of the dataset (unique values)
print(df['dataset'].unique())





Index(['name', 'seed', 'tags', 'dataset', 'epsilon', 'wandb_log',
       'base_learner', 'online_model', '_step', '_timestamp',
       'preview_duration', 'average_records_per_trip', '_wandb', '_runtime',
       'RMSE', 'online_models', 'learning_model', 'R2', 'MAE',
       'average_ahead_records', 'TMI'],
      dtype='object')
['Teconer_Downtown_10K' 'Teconer_Jan_100K' 'Teconer_Jan_1M'
 'Teconer_Downtown_100K' 'Teconer_Jan_10K' 'Teconer_Downtown_1M']


In [88]:
%matplotlib qt



datasets = list(df['dataset'].unique())

for dataset in datasets:
    df_ = df[(df['dataset'] == dataset)]
    f, axs = plt.subplots(1, 3, figsize=(15, 5))
    for i, metric in enumerate(['R2', 'RMSE', 'MAE']):
        # plot barplot of R2 scores for different epsilon values and different preview durations
        sns.set_theme(style="whitegrid")
        # Try using .loc[row_indexer,col_indexer] = value instead
        sns.barplot(data=df_, x="epsilon", y=metric, hue="preview_duration", errorbar="sd", palette="coolwarm", ax=axs[i])
        axs[i].set_xlabel("epsilon")
        axs[i].set_ylabel(metric)
        # rename legend title
        axs[i].legend(title="Preview [min]")
    f.set_size_inches(12, 5)
    f.suptitle(dataset)
    plt.tight_layout()
    
plt.show()





In [90]:
%matplotlib qt


# print average_ahead_records for each dataset, and preview_duration in a table
df_ = df.groupby(['dataset', 'preview_duration'])['average_ahead_records'].mean().reset_index()
df_ = df_.pivot(index='dataset', columns='preview_duration', values='average_ahead_records')
df_ = df_.fillna(0)
# df_ = df_.sort_values(by='dataset', ascending=False)
# sort df_ based on the dataset names (given in the list)
df_ = df_.reindex(['Teconer_Jan_10K', 'Teconer_Jan_100K', 'Teconer_Jan_1M','Teconer_Downtown_10K', 'Teconer_Downtown_100K', 'Teconer_Downtown_1M'])
df_ = df_.round(2)
sns.heatmap(df_, annot=True, cmap="coolwarm", fmt=".2f", cbar=False)
plt.title('Average Number of Requested Preview Points')
plt.xlabel('Preview [min]')
plt.ylabel('Dataset')
plt.tight_layout()
plt.show()



In [98]:
%matplotlib qt


# lineplot average trip duration for each dataset
# only records that have Downtown in their dataset name
# df_ = df
df_ = df[df['dataset'].str.contains("Downtown")].groupby('dataset')['average_records_per_trip'].mean().reset_index()
df_ = df_.sort_values(by='average_records_per_trip', ascending=False)
sns.lineplot(data=df_, x='dataset', y='average_records_per_trip', palette='coolwarm')
plt.xticks(rotation=45)
plt.title('Average Number of Records per Trip')
plt.ylabel('Average Number of Records per Trip')
plt.tight_layout()
plt.show()


/var/folders/f2/z3hnbbhd2h75ldsdvjnzt20m0000gn/T/ipykernel_93532/3507221662.py:9: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.lineplot(data=df_, x='dataset', y='average_records_per_trip', palette='coolwarm')
